In [ ]:
# ==========================================
# Notebook Integrado: Análise de Mata Ciliar com Super-Resolução ESAOpenSR
# ==========================================
# Este notebook é otimizado para análise de mata ciliar:
# - Foco em faixas estreitas ao longo de rios (30-100m)
# - Resolução 2.5m para detectar degradação pontual
# - Análise longitudinal da cobertura vegetal
# - Detecção de pontos críticos de desmatamento
# - Pipeline: AOI → Busca → Super-resolução → NDVI → Pontos críticos

# 1) Instalar pacotes necessários
!pip install mamba-ssm --no-build-isolation -q
!pip install sen2sr mlstac git+https://github.com/ESDS-Leipzig/cubo.git -q
!pip install opensr-model omegaconf rasterio rioxarray geopandas shapely pystac_client planetary-computer torch matplotlib numpy

# 2) Imports
import mlstac
import torch
import cubo
import matplotlib.pyplot as plt
import sen2sr
import rasterio
import numpy as np
import geopandas as gpd
from shapely.geometry import mapping, Point
from pystac_client import Client
import planetary_computer as pc
import json
from rasterio.transform import xy
from rasterio.mask import mask

# 3) Configurações
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")
if device.type != "cuda":
    print("AVISO: LDSR-S2 funciona melhor com GPU. Continuando com CPU...")

# 4) Funções auxiliares
def save_tensor_as_geotiff(tensor, attrs, out_path, super_resolved=False, sr_factor=4):
    """
    Save a PyTorch tensor as a georeferenced GeoTIFF using metadata in attrs.
    """
    if hasattr(tensor, "cpu"):
        tensor = tensor.cpu().numpy()

    # Scale and clip
    arr = (tensor * 10000).clip(0, 10000).astype(np.uint16)

    # Original georef info
    pixel_size = attrs["resolution"]
    edge_size = attrs["edge_size"]
    central_x = attrs["central_x"]
    central_y = attrs["central_y"]
    epsg = attrs["epsg"]

    # Bounding box remains the same
    total_extent = edge_size * pixel_size
    half_extent = total_extent / 2
    ul_x = central_x - half_extent
    ul_y = central_y + half_extent

    # If SR, update pixel size only (dimensions are already upsampled)
    if super_resolved:
        pixel_size = pixel_size / sr_factor

    # Define geotransform
    transform = rasterio.transform.from_origin(ul_x, ul_y, pixel_size, pixel_size)

    # Save
    with rasterio.open(
        out_path,
        "w",
        driver="GTiff",
        height=arr.shape[1],
        width=arr.shape[2],
        count=arr.shape[0],
        dtype=arr.dtype,
        crs=f"EPSG:{epsg}",
        transform=transform,
    ) as dst:
        dst.write(arr)

def plot_lr_sr_comparison(low_resolution, super_resolution):
    """Plot comparison between low resolution and super resolution images"""
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    lr = low_resolution
    sr = super_resolution

    RGB = [2, 1, 0]  # B04, B03, B02
    NIR = [6]  # B08 (índice para 10 bandas)

    # Top left: LR RGB
    axes[0, 0].imshow(torch.clamp(low_resolution[RGB, :, :] * 3.5, 0, 1).permute(1, 2, 0).cpu())
    axes[0, 0].set_title("LR RGB (10m)")
    axes[0, 0].axis('off')

    # Top right: LR NIR
    axes[0, 1].imshow(torch.clamp(low_resolution[NIR, :, :] * 2, 0, 1).permute(1, 2, 0).cpu())
    axes[0, 1].set_title("LR NIR (10m)")
    axes[0, 1].axis('off')

    # Bottom left: SR RGB
    axes[1, 0].imshow(torch.clamp(super_resolution[RGB, :, :] * 3.5, 0, 1).permute(1, 2, 0).cpu())
    axes[1, 0].set_title("SR RGB (2.5m)")
    axes[1, 0].axis('off')

    # Bottom right: SR NIR
    axes[1, 1].imshow(torch.clamp(super_resolution[NIR, :, :] * 2, 0, 1).permute(1, 2, 0).cpu())
    axes[1, 1].set_title("SR NIR (2.5m)")
    axes[1, 1].axis('off')

    plt.tight_layout()
    plt.show()

print("Setup completo! Pronto para carregar AOI e processar imagens.")


In [ ]:
# ==========================================
# ETAPA 1: Carregar e Preparar AOI
# ==========================================

# Carregar AOI (export.geojson)
print("Carregando AOI...")
aoi_gdf = gpd.read_file("export.geojson")
aoi_gdf = aoi_gdf.to_crs("EPSG:4326")
print(f"AOI carregada: {aoi_gdf.total_bounds}")
print(f"Número de geometrias: {len(aoi_gdf)}")
print(f"Tipo de geometria: {aoi_gdf.geometry.geom_type.iloc[0] if len(aoi_gdf) > 0 else 'N/A'}")

# Preparar geometria da AOI
print("Preparando geometria da AOI...")
aoi_gdf_projected = aoi_gdf.to_crs("EPSG:3857")  # Web Mercator
aoi_geom_projected = aoi_gdf_projected.buffer(100).union_all()  # 100m buffer
aoi_gdf_temp = gpd.GeoDataFrame([1], geometry=[aoi_geom_projected], crs="EPSG:3857")
aoi_geom = aoi_gdf_temp.to_crs("EPSG:4326").geometry.iloc[0]

# Corrigir geometria se necessário
if not aoi_geom.is_valid:
    print("Geometria inválida detectada, corrigindo...")
    aoi_geom = aoi_geom.buffer(0)
    print(f"Geometria corrigida - válida: {aoi_geom.is_valid}")

print(f"Geometria final: {aoi_geom.geom_type}")
print(f"   Bounds: {aoi_geom.bounds}")

# Calcular centro da AOI para o Cubo
centroid = aoi_geom.centroid
LATITUDE = centroid.y
LONGITUDE = centroid.x
print(f"Centro da AOI: ({LATITUDE:.6f}, {LONGITUDE:.6f})")


In [ ]:
# ==========================================
# ETAPA 2B: Carregar Modelo ESAOpenSR
# ==========================================

print("Carregando modelo ESAOpenSR para processamento por tiles...")

# Download do modelo LDSR-S2 (ca 1.45GB)
print("Baixando modelo LDSR-S2...")
mlstac.download(
    file="https://huggingface.co/tacofoundation/RS-SR-LTDF/resolve/main/main/mlm.json",
    output_dir="model/LDSRS2-SEN2SR/"
)

# Instanciar o modelo compilado
print("Carregando modelo...")
model = mlstac.load("model/LDSRS2-SEN2SR/").compiled_model(device=device)
model = model.to(device)
print(f"Modelo carregado no dispositivo: {device}")

# ==========================================
# ETAPA 2C: Buscar Dados para AOI Completa com Buffer
# ==========================================

print("Configurando processamento para AOI completa...")

# Calcular bounds da AOI com buffer
aoi_bounds = aoi_gdf.total_bounds
buffer_distance = 0.01  # ~1km em graus (ajustar conforme necessário)

# Aplicar buffer aos bounds
minx, miny, maxx, maxy = aoi_bounds
minx -= buffer_distance
miny -= buffer_distance  
maxx += buffer_distance
maxy += buffer_distance

print(f"Bounds da AOI original: {aoi_bounds}")
print(f"Bounds com buffer: ({minx:.6f}, {miny:.6f}, {maxx:.6f}, {maxy:.6f})")

# Calcular centro da área com buffer
center_lat = (miny + maxy) / 2
center_lon = (minx + maxx) / 2

print(f"Centro da área com buffer: ({center_lat:.6f}, {center_lon:.6f})")

# Calcular tamanho da área em km
import math
lat_km = (maxy - miny) * 111.32  # 1 grau ≈ 111.32 km
lon_km = (maxx - minx) * 111.32 * math.cos(math.radians(center_lat))

print(f"Dimensões da área: {lat_km:.2f}km × {lon_km:.2f}km")

# Calcular número de tiles necessários
tile_size_km = 1.28  # Tamanho de cada tile em km
tiles_lat = int(math.ceil(lat_km / tile_size_km))
tiles_lon = int(math.ceil(lon_km / tile_size_km))

print(f"Tiles necessários: {tiles_lat} × {tiles_lon} = {tiles_lat * tiles_lon} tiles")
print(f"   Cada tile: {tile_size_km}km × {tile_size_km}km (128×128 pixels)")

# Configurações para processamento por tiles
TILE_SIZE = 128  # Pixels por tile
RESOLUTION = 10  # Resolução original em metros
SR_FACTOR = 4    # Fator de super-resolução
FINAL_RESOLUTION = RESOLUTION / SR_FACTOR  # 2.5m

print(f"Configurações:")
print(f"   • Tamanho do tile: {TILE_SIZE}×{TILE_SIZE} pixels")
print(f"   • Resolução original: {RESOLUTION}m")
print(f"   • Resolução final: {FINAL_RESOLUTION}m")
print(f"   • Fator SR: {SR_FACTOR}x")

# Lista para armazenar tiles
tile_centers = []

# Gerar centros dos tiles
for i in range(tiles_lat):
    for j in range(tiles_lon):
        # Calcular centro do tile
        tile_miny = miny + (i * tile_size_km / 111.32)
        tile_maxy = miny + ((i + 1) * tile_size_km / 111.32)
        tile_minx = minx + (j * tile_size_km / (111.32 * math.cos(math.radians(center_lat))))
        tile_maxx = minx + ((j + 1) * tile_size_km / (111.32 * math.cos(math.radians(center_lat))))
        
        tile_center_lat = (tile_miny + tile_maxy) / 2
        tile_center_lon = (tile_minx + tile_maxx) / 2
        
        tile_centers.append({
            'lat': tile_center_lat,
            'lon': tile_center_lon,
            'tile_id': f"tile_{i}_{j}",
            'row': i,
            'col': j
        })

print(f"{len(tile_centers)} centros de tiles calculados")
print("Próximo passo: Processar cada tile individualmente")


In [ ]:
# ==========================================
# ETAPA 2D: Configurar Processamento (Opcional - Teste com Poucos Tiles)
# ==========================================

# Para teste rápido, processar apenas alguns tiles
# Comentar as linhas abaixo para processar todos os tiles
print("Configurando processamento de teste...")

# Limitar número de tiles para teste (opcional)
MAX_TILES_TEST = 20  # Processar apenas os primeiros 20 tiles
if len(tile_centers) > MAX_TILES_TEST:
    print(f"Modo teste: Processando apenas {MAX_TILES_TEST} tiles de {len(tile_centers)}")
    tile_centers = tile_centers[:MAX_TILES_TEST]
    print(f"   Tiles selecionados: {[t['tile_id'] for t in tile_centers]}")
else:
    print(f"Processando todos os {len(tile_centers)} tiles")

print(f"Configuração concluída: {len(tile_centers)} tiles para processar")
print("Para processar todos os tiles, comente as linhas de MAX_TILES_TEST")


In [ ]:
# ==========================================
# ETAPA 2E: Processar Tiles em Lote
# ==========================================

print("Processando tiles em lote...")

# Lista para armazenar resultados de todos os tiles
all_tiles_results = []
all_ndvi_tiles = []
all_critical_points = []

# Configurações de data
START_DATE = "2023-01-01"
END_DATE = "2023-12-31"

# Processar cada tile
for idx, tile_info in enumerate(tile_centers):
    print(f"   Processando {tile_info['tile_id']} ({idx+1}/{len(tile_centers)})...")
    print(f"   Centro: ({tile_info['lat']:.6f}, {tile_info['lon']:.6f})")
    
    try:
        # Buscar dados para este tile
        da_tile = cubo.create(
            lat=tile_info['lat'],
            lon=tile_info['lon'],
            collection="sentinel-2-l2a",
            bands=["B02", "B03", "B04", "B05", "B06", "B07", "B08", "B8A", "B11", "B12"],
            start_date=START_DATE,
            end_date=END_DATE,
            edge_size=TILE_SIZE,
            resolution=RESOLUTION
        )
        
        if len(da_tile) == 0:
            print(f"   Nenhuma imagem encontrada para este tile")
            continue
            
        # Escolher a melhor imagem (menor cobertura de nuvem) se disponível
        IMAGE_INDEX = 0
        try:
            cloud_cov = [img.attrs.get("cloud_coverage", 100.0) for img in da_tile]
            IMAGE_INDEX = int(np.nanargmin(np.array(cloud_cov)))
        except Exception:
            IMAGE_INDEX = 0

        original_s2_numpy = (da_tile[IMAGE_INDEX].compute().to_numpy()).astype("float32")
        low_resolution = torch.from_numpy(original_s2_numpy).float().to(device)
        low_resolution = low_resolution / 10_000  # Normalizar
        
        print(f"   Dados carregados: {low_resolution.shape} | idx={IMAGE_INDEX}")
        
        # Aplicar super-resolução
        print(f"   Aplicando super-resolução...")
        super_resolution = model(low_resolution.unsqueeze(0)).squeeze(0)
        
        print(f"   Super-resolução concluída: {super_resolution.shape}")
        
        # Calcular NDVI
        red_band = super_resolution[2]  # B04 (Red)
        nir_band = super_resolution[6]  # B08 (NIR)
        
        denominator = nir_band + red_band
        ndvi = torch.where(
            denominator != 0,
            (nir_band - red_band) / denominator,
            torch.tensor(float('nan'))
        )
        
        # Converter para numpy
        ndvi_np = ndvi.cpu().numpy()
        
        print(f"   NDVI calculado: {ndvi_np.shape}")
        print(f"   NDVI stats: min={np.nanmin(ndvi_np):.3f}, max={np.nanmax(ndvi_np):.3f}, mean={np.nanmean(ndvi_np):.3f}")
        
        # Armazenar resultados
        tile_result = {
            'tile_id': tile_info['tile_id'],
            'row': tile_info['row'],
            'col': tile_info['col'],
            'center_lat': tile_info['lat'],
            'center_lon': tile_info['lon'],
            'ndvi_data': ndvi_np,
            'super_resolution': super_resolution.cpu().numpy(),
            'low_resolution': low_resolution.cpu().numpy(),
            'attrs': da_tile[IMAGE_INDEX].attrs
        }
        
        all_tiles_results.append(tile_result)
        all_ndvi_tiles.append(ndvi_np)
        
        print(f"   Tile {tile_info['tile_id']} processado com sucesso!")
        
    except Exception as e:
        print(f"   Erro ao processar tile {tile_info['tile_id']}: {str(e)}")
        continue

print(f"   Processamento concluído!")
print(f"   • Tiles processados: {len(all_tiles_results)}/{len(tile_centers)}")
print(f"   • Taxa de sucesso: {len(all_tiles_results)/len(tile_centers)*100:.1f}%")

if len(all_tiles_results) > 0:
    print(f"   • Dados NDVI disponíveis para mosaico")
    print(f"   • Próximo passo: Criar mosaico e analisar pontos críticos")
else:
    print(f"   Nenhum tile foi processado com sucesso")


In [ ]:
# ==========================================
# ETAPA 2F: Criar Mosaico e Analisar Pontos Críticos
# ==========================================

if len(all_tiles_results) > 0:
    print("Criando mosaico NDVI...")
    
    # Calcular dimensões do mosaico
    max_row = max(tile['row'] for tile in all_tiles_results)
    max_col = max(tile['col'] for tile in all_tiles_results)
    
    # Tamanho de cada tile super-resolvido
    tile_sr_size = 512  # 128 * 4 (fator de super-resolução)
    
    # Dimensões do mosaico
    mosaic_height = (max_row + 1) * tile_sr_size
    mosaic_width = (max_col + 1) * tile_sr_size
    
    print(f"Dimensões do mosaico: {mosaic_height} × {mosaic_width} pixels")
    print(f"   • Tiles por linha: {max_col + 1}")
    print(f"   • Tiles por coluna: {max_row + 1}")
    print(f"   • Resolução final: {FINAL_RESOLUTION}m")
    
    # Criar mosaico NDVI
    ndvi_mosaic = np.full((mosaic_height, mosaic_width), np.nan)
    
    # Preencher mosaico com dados dos tiles
    for tile in all_tiles_results:
        row_start = tile['row'] * tile_sr_size
        row_end = row_start + tile_sr_size
        col_start = tile['col'] * tile_sr_size
        col_end = col_start + tile_sr_size
        
        ndvi_mosaic[row_start:row_end, col_start:col_end] = tile['ndvi_data']
        
        print(f"   Tile {tile['tile_id']} adicionado ao mosaico")
    
    print(f"Mosaico criado com sucesso!")
    print(f"   • Dimensões: {ndvi_mosaic.shape}")
    print(f"   • Pixels válidos: {np.sum(~np.isnan(ndvi_mosaic))}")
    print(f"   • NDVI stats: min={np.nanmin(ndvi_mosaic):.3f}, max={np.nanmax(ndvi_mosaic):.3f}, mean={np.nanmean(ndvi_mosaic):.3f}")
    
    # Visualizar mosaico
    print("Visualizando mosaico NDVI...")
    plt.figure(figsize=(15, 10))
    plt.imshow(ndvi_mosaic, cmap='RdYlGn', vmin=-1, vmax=1)
    plt.colorbar(label='NDVI')
    plt.title(f'Mosaico NDVI - Resolução {FINAL_RESOLUTION}m {len(all_tiles_results)} tiles processados')
    plt.xlabel('Pixels')
    plt.ylabel('Pixels')
    plt.show()
    
    # Analisar pontos críticos no mosaico
    print("Analisando pontos críticos no mosaico...")
    
    # Identificar pixels críticos
    critical_pixels = np.where(ndvi_mosaic < 0.3)  # NDVI < 0.3: desmatamento severo
    moderate_pixels = np.where((ndvi_mosaic >= 0.3) & (ndvi_mosaic < 0.5))  # Degradação moderada
    
    print(f"   • Pixels de desmatamento severo (NDVI < 0.3): {len(critical_pixels[0])}")
    print(f"   • Pixels de degradação moderada (0.3 ≤ NDVI < 0.5): {len(moderate_pixels[0])}")
    print(f"   • Total de pixels críticos: {len(critical_pixels[0]) + len(moderate_pixels[0])}")
    
    # Calcular percentual de área crítica
    total_valid_pixels = np.sum(~np.isnan(ndvi_mosaic))
    critical_percentage = (len(critical_pixels[0]) + len(moderate_pixels[0])) / total_valid_pixels * 100
    
    print(f"   • Percentual de área crítica: {critical_percentage:.2f}%")
    
    # Visualizar pontos críticos
    print("Visualizando pontos críticos...")
    fig, axes = plt.subplots(1, 2, figsize=(20, 8))
    
    # Mosaico completo
    im1 = axes[0].imshow(ndvi_mosaic, cmap='RdYlGn', vmin=-1, vmax=1)
    axes[0].set_title(f'Mosaico NDVI Completo Resolução: {FINAL_RESOLUTION}m')
    axes[0].set_xlabel('Pixels')
    axes[0].set_ylabel('Pixels')
    plt.colorbar(im1, ax=axes[0], label='NDVI')
    
    # Pontos críticos
    critical_mask = np.zeros_like(ndvi_mosaic)
    critical_mask[critical_pixels] = 1  # Desmatamento severo
    critical_mask[moderate_pixels] = 0.5  # Degradação moderada
    
    im2 = axes[1].imshow(critical_mask, cmap='Reds', vmin=0, vmax=1)
    axes[1].set_title(f'Pontos Críticos Vermelho: Desmatamento Severo Rosa: Degradação Moderada')
    axes[1].set_xlabel('Pixels')
    axes[1].set_ylabel('Pixels')
    plt.colorbar(im2, ax=axes[1], label='Severidade')
    
    plt.tight_layout()
    plt.show()
    
    print("Análise do mosaico concluída!")
    print(f"   • Área total analisada: {mosaic_height * mosaic_width * (FINAL_RESOLUTION**2) / 1_000_000:.2f} km²")
    print(f"   • Resolução espacial: {FINAL_RESOLUTION}m")
    print(f"   • Tiles processados: {len(all_tiles_results)}")
    
else:
    print("Nenhum tile foi processado. Não é possível criar o mosaico.")


In [ ]:
# ==========================================
# ETAPA 2I: Servir tiles com TiTiler localmente (opcional)
# ==========================================

# Este bloco levanta um servidor TiTiler local para visualizar tiles do COG
# Execute e abra a URL impressa. Para produção, implante o TiTiler separadamente.

import os

print("Iniciando TiTiler local (uvicorn)...")
print("Abra em: http://127.0.0.1:8000")
print("Endpoints úteis:")
print(" - /docs (Swagger)")
print(" - /cog/tiles/{z}/{x}/{y}.png?url=http://127.0.0.1:8000/static/ndvi.tif")

# Copiar COG para pasta static e iniciar servidor simples
os.makedirs("static", exist_ok=True)
import shutil
shutil.copyfile("ndvi_mosaic_super_resolved_cog.tif", "static/ndvi.tif")

# Iniciar TiTiler app
from titiler.application.main import app
import uvicorn

uvicorn.run(app, host="127.0.0.1", port=8000)


In [ ]:
# ==========================================
# ETAPA 2G: Salvar Resultados do Mosaico
# ==========================================

if len(all_tiles_results) > 0:
    print("Salvando resultados do mosaico...")
    
    # 1) Salvar mosaico NDVI como GeoTIFF
    print("Salvando mosaico NDVI como GeoTIFF...")
    
    # Calcular transformação geográfica para o mosaico
    # Usar o primeiro tile como referência
    first_tile = all_tiles_results[0]
    attrs = first_tile['attrs']
    
    # Calcular bounds do mosaico
    mosaic_bounds = aoi_gdf.total_bounds
    buffer_distance = 0.01
    
    minx, miny, maxx, maxy = mosaic_bounds
    minx -= buffer_distance
    miny -= buffer_distance
    maxx += buffer_distance
    maxy += buffer_distance
    
    # Criar transformação correta no CRS nativo dos tiles (metros)
    from rasterio.transform import Affine

    # Tamanho de pixel super-resolvido em metros
    px = attrs["resolution"] / SR_FACTOR  # 10m/4 = 2.5m

    # Calcular canto superior-esquerdo do mosaico a partir dos tiles
    # Para cada tile, derive o canto superior-esquerdo real usando central_x/central_y
    tile_ul_x = []
    tile_ul_y = []
    for tile in all_tiles_results:
        t_attrs = tile.get("attrs", attrs)
        cx = t_attrs["central_x"]
        cy = t_attrs["central_y"]
        ulx = cx - (tile_sr_size * px) / 2.0
        uly = cy + (tile_sr_size * px) / 2.0
        tile_ul_x.append(ulx)
        tile_ul_y.append(uly)

    ul_x = float(np.min(tile_ul_x))
    ul_y = float(np.max(tile_ul_y))

    # Affine from origin (west, north, pixel size)
    transform = Affine.translation(ul_x, ul_y) * Affine.scale(px, -px)

    # Salvar mosaico NDVI em CRS dos tiles
    with rasterio.open(
        "ndvi_mosaic_super_resolved.tif",
        "w",
        driver="GTiff",
        height=mosaic_height,
        width=mosaic_width,
        count=1,
        dtype=rasterio.float32,
        crs=f"EPSG:{attrs['epsg']}",
        transform=transform,
    ) as dst:
        dst.write(ndvi_mosaic.astype(np.float32), 1)
    
    print("Mosaico NDVI salvo: ndvi_mosaic_super_resolved.tif")
    
    # 2) Salvar relatório do mosaico
    print("Gerando relatório do mosaico...")
    
    mosaic_report = {
        "processamento_mosaico": {
            "data_processamento": str(np.datetime64('now')),
            "aoi_bounds_original": list(aoi_bounds),
            "aoi_bounds_com_buffer": [minx, miny, maxx, maxy],
            "buffer_distance_km": buffer_distance * 111.32,
            "dimensoes_mosaico": [mosaic_height, mosaic_width],
            "resolucao_final": f"{FINAL_RESOLUTION}m",
            "fator_super_resolucao": f"{SR_FACTOR}x",
            "metodo": "ESAOpenSR LDSR-S2 - Processamento por Tiles"
        },
        "tiles_processados": {
            "total_tiles_calculados": len(tile_centers),
            "tiles_processados_com_sucesso": len(all_tiles_results),
            "taxa_sucesso": f"{len(all_tiles_results)/len(tile_centers)*100:.1f}%",
            "tiles_por_linha": max_col + 1,
            "tiles_por_coluna": max_row + 1,
            "tamanho_tile_km": f"{tile_size_km}km × {tile_size_km}km"
        },
        "estatisticas_ndvi": {
            "min": float(np.nanmin(ndvi_mosaic)),
            "max": float(np.nanmax(ndvi_mosaic)),
            "mean": float(np.nanmean(ndvi_mosaic)),
            "std": float(np.nanstd(ndvi_mosaic)),
            "pixels_validos": int(np.sum(~np.isnan(ndvi_mosaic))),
            "pixels_totais": int(ndvi_mosaic.size)
        },
        "analise_critica": {
            "pixels_desmatamento_severo": int(len(critical_pixels[0])),
            "pixels_degradacao_moderada": int(len(moderate_pixels[0])),
            "total_pixels_criticos": int(len(critical_pixels[0]) + len(moderate_pixels[0])),
            "percentual_area_critica": f"{critical_percentage:.2f}%",
            "threshold_desmatamento_severo": "NDVI < 0.3",
            "threshold_degradacao_moderada": "0.3 ≤ NDVI < 0.5"
        },
        "area_analisada": {
            "area_total_km2": f"{mosaic_height * mosaic_width * (FINAL_RESOLUTION**2) / 1_000_000:.2f}",
            "resolucao_espacial": f"{FINAL_RESOLUTION}m",
            "cobertura": "AOI completa com buffer"
        },
        "arquivos_gerados": [
            "ndvi_mosaic_super_resolved.tif",
            "relatorio_mosaico_super_resolucao.json"
        ]
    }
    
    with open("relatorio_mosaico_super_resolucao.json", "w") as f:
        json.dump(mosaic_report, f, indent=2)
    
    print("Relatório salvo: relatorio_mosaico_super_resolucao.json")
    
    # 3) Salvar lista de tiles processados
    print("Salvando lista de tiles processados...")
    
    tiles_info = []
    for tile in all_tiles_results:
        ndvi_arr = tile['ndvi_data']
        valid_mask = ~np.isnan(ndvi_arr)
        has_valid = np.any(valid_mask)
        ndvi_min = float(np.nanmin(ndvi_arr)) if has_valid else None
        ndvi_max = float(np.nanmax(ndvi_arr)) if has_valid else None
        ndvi_mean = float(np.nanmean(ndvi_arr)) if has_valid else None
        tiles_info.append({
            "tile_id": tile['tile_id'],
            "row": tile['row'],
            "col": tile['col'],
            "center_lat": tile['center_lat'],
            "center_lon": tile['center_lon'],
            "ndvi_stats": {
                "min": ndvi_min,
                "max": ndvi_max,
                "mean": ndvi_mean,
                "valid_pixels": int(np.sum(valid_mask))
            }
        })
    
    with open("tiles_processados_info.json", "w") as f:
        json.dump(tiles_info, f, indent=2)
    
    print("Lista de tiles salva: tiles_processados_info.json")
    
    print("   PROCESSAMENTO DO MOSAICO CONCLUÍDO COM SUCESSO!")
    print("=" * 60)
    print(f"Resumo dos resultados:")
    print(f"   • {len(all_tiles_results)} tiles processados")
    print(f"   • Mosaico: {mosaic_height} × {mosaic_width} pixels")
    print(f"   • Resolução: {FINAL_RESOLUTION}m")
    print(f"   • Área analisada: {mosaic_height * mosaic_width * (FINAL_RESOLUTION**2) / 1_000_000:.2f} km²")
    print(f"   • Pontos críticos: {len(critical_pixels[0]) + len(moderate_pixels[0])} pixels")
    print(f"   • Percentual crítico: {critical_percentage:.2f}%")
    print(f"   • 3 arquivos gerados")
    print("=" * 60)
    
else:
    print("Nenhum tile foi processado. Não há resultados para salvar.")
